In [468]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from pprint import pprint
import json
from time import sleep

header = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36', 
         'Accept':'*/*'}

time_to_sleep_when_captcha = 10
main_link_hh = 'https://hh.ru'
main_link_sj = 'https://russia.superjob.ru'
job_name = str(input('Введите должность: '))
# job_name_hh = 'Data+scientist'
# job_name_sj = 'data-scientist'

url_hh = f'/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text={job_name}&page=0'
url_sj = f'/vakansii/{job_name}.html'

In [469]:
def get_new_soup_hh(url_hh):
    response_hh = requests.get(main_link_hh + url_hh, headers=header).text
    soup_hh = bs(response_hh,'lxml')
    return soup_hh

def get_new_vacancy_list_hh(soup_hh):
    vacancy_block_hh = soup_hh.find('div', {'class':'vacancy-serp'})
    vacancy_list_hh = vacancy_block_hh.find_all('div', {'class':'vacancy-serp-item'})
    return vacancy_list_hh

In [470]:
def get_new_soup_sj(url_sj):
    response_sj = requests.get(main_link_sj + url_sj, headers=header).text
    soup_sj = bs(response_sj,'lxml')
    return soup_sj

def get_new_vacancy_list_sj(soup_sj):
    vacancy_list_sj = soup_sj.find_all('div', {'class':'_3zucV _1fMKr undefined _1NAsu'})
    return vacancy_list_sj

In [471]:
vacancies = []

def get_vacancies_hh(v_list, soup_hh):
    for vacancy in v_list:
        vacancy_data = {}
        vacancy_name = vacancy.find('a', {'class':'bloko-link'}).getText()
        vacancy_salary = vacancy.find('div', {'class':'vacancy-serp-item__sidebar'}).getText()
        vacancy_link = vacancy.find('a', {'class':'bloko-link'})['href']
        vacancy_site = main_link_hh
        vacancy_data['name'] = vacancy_name
        if len(vacancy_salary) < 1:
            vacancy_data['salary'] = None
        else:
            vacancy_data['salary'] = vacancy_salary
        vacancy_data['link'] = vacancy_link
        vacancy_data['site'] = vacancy_site
        vacancies.append(vacancy_data)
    
    sleep(time_to_sleep_when_captcha)
    next_button_link_hh = soup_hh.find('a', {'class':'HH-Pager-Controls-Next'})['href']
    return next_button_link_hh

def get_vacancies_sj(v_list, soup_sj):
    for vacancy in v_list:
        vacancy_data = {}
        vacancy_name = vacancy.find('div', {'class':'_3mfro'}).getText()
        vacancy_salary = vacancy.find('span', {'class':'_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'}).getText()
        vacancy_link = main_link_sj + vacancy.find('a', {'class':'icMQ_ _6AfZ9 f-test-link-Genetik-analitik_dannyh_Data_scientist _2JivQ _1UJAN'})['href']
        vacancy_site = main_link_sj
        vacancy_data['name'] = vacancy_name
        if len(vacancy_salary) < 1:
            vacancy_data['salary'] = None
        else:
            vacancy_data['salary'] = vacancy_salary
        vacancy_data['link'] = vacancy_link
        vacancy_data['site'] = vacancy_site
        vacancies.append(vacancy_data)
    
    sleep(time_to_sleep_when_captcha)
    next_button_link_sj = main_link_sj + soup_sj.find('a', {'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'})['href']
    return next_button_link_sj

In [472]:
while True:
    soup_hh = get_new_soup_hh(url_hh)
    vacancy_list_hh = get_new_vacancy_list_hh(soup_hh)
    try:
        url_hh = get_vacancies_hh(vacancy_list_hh, soup_hh)
    except TypeError:
        break

/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+scientist&page=1
/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+scientist&page=2
/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+scientist&page=3
/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+scientist&page=4
/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+scientist&page=5
/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&text=Data+scientist&page=6


In [473]:
while True:
    soup_sj = get_new_soup_sj(url_sj)
    vacancy_list_sj = get_new_vacancy_list_sj(soup_sj)
    try:
        url_sj = get_vacancies_sj(vacancy_list_sj, soup_sj)
    except TypeError:
        break

{'name': 'Генетик-аналитик данных / Data scientist', 'salary': 'По договорённости', 'link': 'https://russia.superjob.ru/vakansii/genetik-analitik-dannyh-33864238.html', 'site': 'https://russia.superjob.ru'}


In [474]:
vacancies_df = pd.DataFrame(vacancies)
vacancies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 4 columns):
name      314 non-null object
salary    45 non-null object
link      314 non-null object
site      314 non-null object
dtypes: object(4)
memory usage: 9.9+ KB


In [475]:
vacancies_df.head(10)

,name,salary,link,site
0,Data Scientist,None,https://hh.ru/vacancy/37645302?query=Data%20sc...,https://hh.ru
1,Senior Data Scientist,None,https://hh.ru/vacancy/37645039?query=Data%20sc...,https://hh.ru
2,Python Data Scientist,от 150 000 руб.,https://hh.ru/vacancy/37521387?query=Data%20sc...,https://hh.ru
3,Senior Data Scientist,от 300 000 руб.,https://hh.ru/vacancy/37294434?query=Data%20sc...,https://hh.ru
4,Data Scientist,от 130 000 руб.,https://hh.ru/vacancy/37569615?query=Data%20sc...,https://hh.ru
5,Data Scientist,120 000-180 000 руб.,https://hh.ru/vacancy/37635578?query=Data%20sc...,https://hh.ru
6,Data Scientist,None,https://hh.ru/vacancy/37662353?query=Data%20sc...,https://hh.ru
7,Data Scientist,None,https://hh.ru/vacancy/37706122?query=Data%20sc...,https://hh.ru
8,Representation Learning Senior Research Scientist,3 000-8 000 USD,https://hh.ru/vacancy/37143803?query=Data%20sc...,https://hh.ru
9,Аналитик-математик (Data scientist),80 000-100 000 руб.,https://hh.ru/vacancy/37698748?query=Data%20sc...,https://hh.ru
